# Ragas API Client

> Python client to api.ragas.io

In [ ]:
#| default_exp backends.ragas_api_client

In [ ]:
RAGAS_APP_TOKEN = "api_key"
RAGAS_API_ENDPOINT = "https://api.dev.app.ragas.io"

In [ ]:
#| export
import httpx
import asyncio
import typing as t
from pydantic import BaseModel, Field
from fastcore.utils import patch

In [ ]:
#| export
from ragas_experimental.exceptions import (
    DatasetNotFoundError, DuplicateDatasetError,
    ProjectNotFoundError, DuplicateProjectError,
    ExperimentNotFoundError, DuplicateExperimentError
)

In [ ]:
#| export
class RagasApiClient():
    """Client for the Ragas Relay API."""

    def __init__(self, base_url: str, app_token: t.Optional[str] = None):
        """Initialize the Ragas API client.
        
        Args:
            base_url: Base URL for the API (e.g., "http://localhost:8087")
            app_token: API token for authentication
        """
        if not app_token:
            raise ValueError("app_token must be provided")

        self.base_url = f"{base_url.rstrip('/')}/api/v1"
        self.app_token = app_token

    async def _request(
        self,
        method: str,
        endpoint: str,
        params: t.Optional[t.Dict] = None,
        json_data: t.Optional[t.Dict] = None,
    ) -> t.Dict:
        """Make a request to the API.
        
        Args:
            method: HTTP method (GET, POST, PATCH, DELETE)
            endpoint: API endpoint path
            params: Query parameters
            json_data: JSON request body
            
        Returns:
            The response data from the API
        """
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        headers = {"X-App-Token": self.app_token}

        async with httpx.AsyncClient() as client:
            response = await client.request(
                method=method, url=url, params=params, json=json_data, headers=headers
            )

            data = response.json()

            if response.status_code >= 400 or data.get("status") == "error":
                error_msg = data.get("message", "Unknown error")
                raise Exception(f"API Error ({response.status_code}): {error_msg}")

            return data.get("data")

    #---- Resource Handlers ----
    async def _create_resource(self, path, data):
        """Generic resource creation."""
        return await self._request("POST", path, json_data=data)
        
    async def _list_resources(self, path, **params):
        """Generic resource listing."""
        return await self._request("GET", path, params=params)
        
    async def _get_resource(self, path):
        """Generic resource retrieval."""
        return await self._request("GET", path)
        
    async def _update_resource(self, path, data):
        """Generic resource update."""
        return await self._request("PATCH", path, json_data=data)
        
    async def _delete_resource(self, path):
        """Generic resource deletion."""
        return await self._request("DELETE", path)

In [ ]:
#| export
@patch
async def _get_resource_by_name(
    self: RagasApiClient,
    list_method: t.Callable,
    get_method: t.Callable,
    resource_name: str,
    name_field: str,
    not_found_error: t.Type[Exception],
    duplicate_error: t.Type[Exception],
    resource_type_name: str,
    **list_method_kwargs
) -> t.Dict:
    """Generic method to get a resource by name.
    
    Args:
        list_method: Method to list resources
        get_method: Method to get a specific resource
        resource_name: Name to search for
        name_field: Field name that contains the resource name
        not_found_error: Exception to raise when resource is not found
        duplicate_error: Exception to raise when multiple resources are found
        resource_type_name: Human-readable name of the resource type
        **list_method_kwargs: Additional arguments to pass to list_method
        
    Returns:
        The resource information dictionary
        
    Raises:
        Exception: If resource is not found or multiple resources are found
    """
    # Initial pagination parameters
    limit = 50  # Number of items per page
    offset = 0  # Starting position
    matching_resources = []
    
    while True:
        # Get a page of resources
        response = await list_method(
            limit=limit,
            offset=offset,
            **list_method_kwargs
        )
        
        items = response.get("items", [])
        
        # If no items returned, we've reached the end
        if not items:
            break
            
        # Collect all resources with the matching name in this page
        for resource in items:
            if resource.get(name_field) == resource_name:
                matching_resources.append(resource)
        
        # Update offset for the next page
        offset += limit
        
        # If we've processed all items (less than limit returned), exit the loop
        if len(items) < limit:
            break
    
    # Check results
    if not matching_resources:
        context = list_method_kwargs.get("project_id", "")
        context_msg = f" in project {context}" if context else ""
        raise not_found_error(
            f"No {resource_type_name} with name '{resource_name}' found{context_msg}"
        )
    
    if len(matching_resources) > 1:
        # Multiple matches found - construct an informative error message
        resource_ids = [r.get("id") for r in matching_resources]
        context = list_method_kwargs.get("project_id", "")
        context_msg = f" in project {context}" if context else ""
        
        raise duplicate_error(
            f"Multiple {resource_type_name}s found with name '{resource_name}'{context_msg}. "
            f"{resource_type_name.capitalize()} IDs: {', '.join(resource_ids)}. "
            f"Please use get_{resource_type_name}() with a specific ID instead."
        )
    
    # Exactly one match found - retrieve full details
    if "project_id" in list_method_kwargs:
        return await get_method(list_method_kwargs["project_id"], matching_resources[0].get("id"))
    else:
        return await get_method(matching_resources[0].get("id"))

### Projects

In [ ]:
#| export
#---- Projects ----
@patch
async def list_projects(
    self: RagasApiClient,
    ids: t.Optional[t.List[str]] = None,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List projects."""
    params = {"limit": limit, "offset": offset}

    if ids:
        params["ids"] = ",".join(ids)

    if order_by:
        params["order_by"] = order_by

    if sort_dir:
        params["sort_dir"] = sort_dir

    return await self._list_resources("projects", **params)

@patch
async def get_project(self: RagasApiClient, project_id: str) -> t.Dict:
    """Get a specific project by ID."""
    # TODO: Need get project by title
    return await self._get_resource(f"projects/{project_id}")

@patch
async def create_project(
    self: RagasApiClient, title: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new project."""
    data = {"title": title}
    if description:
        data["description"] = description
    return await self._create_resource("projects", data)

@patch
async def update_project(
    self: RagasApiClient,
    project_id: str,
    title: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing project."""
    data = {}
    if title:
        data["title"] = title
    if description:
        data["description"] = description
    return await self._update_resource(f"projects/{project_id}", data)

@patch
async def delete_project(self: RagasApiClient, project_id: str) -> None:
    """Delete a project."""
    await self._delete_resource(f"projects/{project_id}")


In [ ]:
# Initialize client with your authentication token
client = RagasApiClient(base_url=RAGAS_API_ENDPOINT, app_token=RAGAS_APP_TOKEN)

# List projects
try:
    projects = await client.list_projects(limit=10)
    print(f"Found {len(projects)} projects:")
    for project in projects:
        print(f"- {project['title']} (ID: {project['id']})")
except Exception as e:
    print(f"Error: {e}")

Found 2 projects:
Error: string indices must be integers, not 'str'


In [ ]:
await client.create_project("test project", "test description")

{'id': '26b0e577-8ff8-4014-bc7a-cfc410df3488',
 'title': 'test project',
 'description': 'test description',
 'created_at': '2025-04-10T00:12:34.606398+00:00',
 'updated_at': '2025-04-10T00:12:34.606398+00:00'}

In [ ]:
await client.list_projects()

{'items': [{'id': '1ef0843b-231f-4a2c-b64d-d39bcee9d830',
   'title': 'yann-lecun-wisdom',
   'description': 'Yann LeCun Wisdom',
   'created_at': '2025-04-15T03:27:08.962384+00:00',
   'updated_at': '2025-04-15T03:27:08.962384+00:00'},
  {'id': 'c2d788ec-a602-495b-8ddc-f457ce11b414',
   'title': 'Demo Project',
   'description': None,
   'created_at': '2025-04-12T19:47:10.928422+00:00',
   'updated_at': '2025-04-12T19:47:10.928422+00:00'},
  {'id': '0d465f02-c88f-454e-9ff3-780a001e3e21',
   'title': 'test project',
   'description': 'test description',
   'created_at': '2025-04-12T19:46:36.221385+00:00',
   'updated_at': '2025-04-12T19:46:36.221385+00:00'},
  {'id': '2ae1434c-e700-44a7-9528-7c2f03cfb491',
   'title': 'Demo Project',
   'description': None,
   'created_at': '2025-04-12T19:46:36.157122+00:00',
   'updated_at': '2025-04-12T19:46:36.157122+00:00'},
  {'id': 'adb45ec6-6902-4339-b05f-3b86fd256c7e',
   'title': 'Demo Project',
   'description': None,
   'created_at': '2025-0

In [ ]:
TEST_PROJECT_ID = "a6ccabe0-7b8d-4866-98af-f167a36b94ff"
project = await client.get_project(TEST_PROJECT_ID)

In [ ]:
#| export
@patch
async def get_project_by_name(
    self: RagasApiClient, project_name: str
) -> t.Dict:
    """Get a project by its name.
    
    Args:
        project_name: Name of the project to find
        
    Returns:
        The project information dictionary
        
    Raises:
        ProjectNotFoundError: If no project with the given name is found
        DuplicateProjectError: If multiple projects with the given name are found
    """
    return await self._get_resource_by_name(
        list_method=self.list_projects,
        get_method=self.get_project,
        resource_name=project_name,
        name_field="title",  # Projects use 'title' instead of 'name'
        not_found_error=ProjectNotFoundError,
        duplicate_error=DuplicateProjectError,
        resource_type_name="project"
    )

In [ ]:
await client.get_project_by_name("SuperMe")

{'id': 'a6ccabe0-7b8d-4866-98af-f167a36b94ff',
 'title': 'SuperMe',
 'description': 'SuperMe demo to show the team',
 'created_at': '2025-04-10T03:10:29.153622+00:00',
 'updated_at': '2025-04-10T03:10:29.153622+00:00'}

### Datasets

In [ ]:
#| export

#---- Datasets ----
@patch
async def list_datasets(
    self: RagasApiClient,
    project_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List datasets in a project."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(f"projects/{project_id}/datasets", **params)

@patch
async def get_dataset(self: RagasApiClient, project_id: str, dataset_id: str) -> t.Dict:
    """Get a specific dataset."""
    return await self._get_resource(f"projects/{project_id}/datasets/{dataset_id}")

@patch
async def create_dataset(
    self: RagasApiClient, project_id: str, name: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new dataset in a project."""
    data = {"name": name}
    if description:
        data["description"] = description
    return await self._create_resource(f"projects/{project_id}/datasets", data)

@patch
async def update_dataset(
    self: RagasApiClient,
    project_id: str,
    dataset_id: str,
    name: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing dataset."""
    data = {}
    if name:
        data["name"] = name
    if description:
        data["description"] = description
    return await self._update_resource(f"projects/{project_id}/datasets/{dataset_id}", data)

@patch
async def delete_dataset(self: RagasApiClient, project_id: str, dataset_id: str) -> None:
    """Delete a dataset."""
    await self._delete_resource(f"projects/{project_id}/datasets/{dataset_id}")

In [ ]:
# check project ID
projects = await client.list_projects()
projects["items"][0]["id"], TEST_PROJECT_ID

('1ef0843b-231f-4a2c-b64d-d39bcee9d830',
 'a6ccabe0-7b8d-4866-98af-f167a36b94ff')

In [ ]:
# Create a new dataset
new_dataset = await client.create_dataset(
    projects["items"][0]["id"], "New Dataset", "This is a new dataset"
)
print(f"New dataset created: {new_dataset}")

New dataset created: {'id': '2382037f-906c-45a0-9b9f-702d32903efd', 'name': 'New Dataset', 'description': 'This is a new dataset', 'updated_at': '2025-04-16T03:52:01.91574+00:00', 'created_at': '2025-04-16T03:52:01.91574+00:00', 'version_counter': 0, 'project_id': '1ef0843b-231f-4a2c-b64d-d39bcee9d830'}


In [ ]:
# List datasets in the project
datasets = await client.list_datasets(projects["items"][0]["id"])
print(f"Found {len(datasets)} datasets")

Found 2 datasets


In [ ]:
updated_dataset = await client.update_dataset(
    projects["items"][0]["id"],
    datasets["items"][0]["id"],
    "Updated Dataset",
    "This is an updated dataset",
)
print(f"Updated dataset: {updated_dataset}")

Updated dataset: {'id': '8572180f-fddf-46c5-b943-e6ff6448eb01', 'name': 'Updated Dataset', 'description': 'This is an updated dataset', 'created_at': '2025-04-15T03:28:09.050125+00:00', 'updated_at': '2025-04-16T03:52:09.627448+00:00', 'version_counter': 0, 'project_id': '1ef0843b-231f-4a2c-b64d-d39bcee9d830'}


In [ ]:
# Delete the dataset
await client.delete_dataset(projects["items"][0]["id"], datasets["items"][0]["id"])
print("Dataset deleted")

Dataset deleted


For the time being I've also added another option to get the dataset by name too

In [ ]:
#| export
@patch
async def get_dataset_by_name(
    self: RagasApiClient, project_id: str, dataset_name: str
) -> t.Dict:
    """Get a dataset by its name.
    
    Args:
        project_id: ID of the project
        dataset_name: Name of the dataset to find
        
    Returns:
        The dataset information dictionary
        
    Raises:
        DatasetNotFoundError: If no dataset with the given name is found
        DuplicateDatasetError: If multiple datasets with the given name are found
    """
    return await self._get_resource_by_name(
        list_method=self.list_datasets,
        get_method=self.get_dataset,
        resource_name=dataset_name,
        name_field="name",
        not_found_error=DatasetNotFoundError,
        duplicate_error=DuplicateDatasetError,
        resource_type_name="dataset",
        project_id=project_id
    )

In [ ]:
await client.get_dataset_by_name(project_id=TEST_PROJECT_ID, dataset_name="test")

DuplicateDatasetError: Multiple datasets found with name 'test' in project a6ccabe0-7b8d-4866-98af-f167a36b94ff. Dataset IDs: 9a48d5d1-531f-424f-b2d2-d8f9bcaeec1e, 483477a4-3d00-4010-a253-c92dee3bc092. Please use get_dataset() with a specific ID instead.

### Experiments

In [ ]:
 #| export
#---- Experiments ----
@patch
async def list_experiments(
    self: RagasApiClient,
    project_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List experiments in a project."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(f"projects/{project_id}/experiments", **params)

@patch
async def get_experiment(self: RagasApiClient, project_id: str, experiment_id: str) -> t.Dict:
    """Get a specific experiment."""
    return await self._get_resource(f"projects/{project_id}/experiments/{experiment_id}")

@patch
async def create_experiment(
    self: RagasApiClient, project_id: str, name: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new experiment in a project."""
    data = {"name": name}
    if description:
        data["description"] = description
    return await self._create_resource(f"projects/{project_id}/experiments", data)

@patch
async def update_experiment(
    self: RagasApiClient,
    project_id: str,
    experiment_id: str,
    name: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing experiment."""
    data = {}
    if name:
        data["name"] = name
    if description:
        data["description"] = description
    return await self._update_resource(f"projects/{project_id}/experiments/{experiment_id}", data)

@patch
async def delete_experiment(self: RagasApiClient, project_id: str, experiment_id: str) -> None:
    """Delete an experiment."""
    await self._delete_resource(f"projects/{project_id}/experiments/{experiment_id}")


In [ ]:
# create a new experiment
new_experiment = await client.create_experiment(
    projects["items"][0]["id"], "New Experiment", "This is a new experiment"
)
print(f"New experiment created: {new_experiment}")
# list experiments
experiments = await client.list_experiments(projects["items"][0]["id"])
print(f"Found {len(experiments)} experiments")
# get a specific experiment
experiment = await client.get_experiment(
    projects["items"][0]["id"], experiments["items"][0]["id"]
)
print(f"Experiment: {experiment}")
# update an experiment
updated_experiment = await client.update_experiment(
    projects["items"][0]["id"],
    experiments["items"][0]["id"],
    "Updated Experiment",
    "This is an updated experiment",
)
print(f"Updated experiment: {updated_experiment}")
# delete an experiment
await client.delete_experiment(projects["items"][0]["id"], experiments["items"][0]["id"])
print("Experiment deleted")

New experiment created: {'id': 'b575c5d1-6934-45c0-b67a-fc9a4d7bdba3', 'name': 'New Experiment', 'description': 'This is a new experiment', 'updated_at': '2025-04-10T00:12:39.955229+00:00', 'created_at': '2025-04-10T00:12:39.955229+00:00', 'version_counter': 0, 'project_id': '26b0e577-8ff8-4014-bc7a-cfc410df3488'}
Found 2 experiments
Experiment: {'id': 'b575c5d1-6934-45c0-b67a-fc9a4d7bdba3', 'name': 'New Experiment', 'description': 'This is a new experiment', 'created_at': '2025-04-10T00:12:39.955229+00:00', 'updated_at': '2025-04-10T00:12:39.955229+00:00', 'version_counter': 0, 'project_id': '26b0e577-8ff8-4014-bc7a-cfc410df3488'}
Updated experiment: {'id': 'b575c5d1-6934-45c0-b67a-fc9a4d7bdba3', 'name': 'Updated Experiment', 'description': 'This is an updated experiment', 'created_at': '2025-04-10T00:12:39.955229+00:00', 'updated_at': '2025-04-10T00:12:41.676216+00:00', 'version_counter': 0, 'project_id': '26b0e577-8ff8-4014-bc7a-cfc410df3488'}
Experiment deleted


In [ ]:
await client.list_experiments(TEST_PROJECT_ID)

{'items': [{'id': '78fd6c58-7edf-4239-93d1-4f49185d8e49',
   'name': 'New Experiment',
   'description': 'This is a new experiment',
   'created_at': '2025-03-30T06:31:31.689269+00:00',
   'updated_at': '2025-03-30T06:31:31.689269+00:00',
   'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'},
  {'id': '7c695b58-7fc3-464c-a18b-a96e35f9684d',
   'name': 'New Experiment',
   'description': 'This is a new experiment',
   'created_at': '2025-04-09T17:03:44.340782+00:00',
   'updated_at': '2025-04-09T17:03:44.340782+00:00',
   'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 2,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

In [ ]:
#| export
@patch
async def get_experiment_by_name(
    self: RagasApiClient, project_id: str, experiment_name: str
) -> t.Dict:
    """Get an experiment by its name.
    
    Args:
        project_id: ID of the project containing the experiment
        experiment_name: Name of the experiment to find
        
    Returns:
        The experiment information dictionary
        
    Raises:
        ExperimentNotFoundError: If no experiment with the given name is found
        DuplicateExperimentError: If multiple experiments with the given name are found
    """
    return await self._get_resource_by_name(
        list_method=self.list_experiments,
        get_method=self.get_experiment,
        resource_name=experiment_name,
        name_field="name",
        not_found_error=ExperimentNotFoundError,
        duplicate_error=DuplicateExperimentError,
        resource_type_name="experiment",
        project_id=project_id
    )

In [ ]:
await client.get_experiment_by_name(TEST_PROJECT_ID, "test")

DuplicateExperimentError: Multiple experiments found with name 'test' in project a6ccabe0-7b8d-4866-98af-f167a36b94ff. Experiment IDs: e1ae15aa-2e0e-40dd-902a-0f0e0fd4df69, 52428c79-afdf-468e-82dc-6ef82c5b71d2, 55e14ac3-0037-4909-898f-eee9533a6d3f, 9adfa008-b479-41cf-ba28-c860e01401ea, 233d28c8-6556-49c5-b146-1e001720c214, 6aed5143-3f60-4bf2-bcf2-ecfdb950e992. Please use get_experiment() with a specific ID instead.

### Columns (for datasets)

In [ ]:
#| export
from ragas_experimental.typing import ColumnType

In [ ]:
#| export

#---- Dataset Columns ----
@patch
async def list_dataset_columns(
    self: RagasApiClient,
    project_id: str,
    dataset_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List columns in a dataset."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/datasets/{dataset_id}/columns", **params
    )

@patch
async def get_dataset_column(
    self: RagasApiClient, project_id: str, dataset_id: str, column_id: str
) -> t.Dict:
    """Get a specific column in a dataset."""
    return await self._get_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}"
    )

@patch
async def create_dataset_column(
    self: RagasApiClient,
    project_id: str,
    dataset_id: str,
    id: str,
    name: str,
    type: str,
    col_order: t.Optional[int] = None,
    settings: t.Optional[t.Dict] = None,
) -> t.Dict:
    """Create a new column in a dataset."""
    data = {"id": id, "name": name, "type": type}
    if col_order is not None:
        data["col_order"] = col_order
    if settings:
        data["settings"] = settings
    return await self._create_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns", data
    )

@patch
async def update_dataset_column(
    self: RagasApiClient, project_id: str, dataset_id: str, column_id: str, **column_data
) -> t.Dict:
    """Update an existing column in a dataset."""
    return await self._update_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}",
        column_data,
    )

@patch
async def delete_dataset_column(
    self: RagasApiClient, project_id: str, dataset_id: str, column_id: str
) -> None:
    """Delete a column from a dataset."""
    await self._delete_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}"
    )

In [ ]:
datasets = await client.create_dataset(
    projects["items"][0]["id"],
    "New Dataset for testing columns",
    "This is a new dataset for testing columns",
)
datasets

{'id': 'cc6794e1-3505-4d5c-b403-ca7e55142bbc',
 'name': 'New Dataset for testing columns',
 'description': 'This is a new dataset for testing columns',
 'updated_at': '2025-04-16T18:05:53.249101+00:00',
 'created_at': '2025-04-16T18:05:53.249101+00:00',
 'version_counter': 0,
 'project_id': '3d9b529b-c23f-4e87-8a26-dd1923749aa7'}

In [ ]:
# add a new column to the dataset
new_column = await client.create_dataset_column(
    project_id=projects["items"][0]["id"],
    dataset_id=datasets["id"],
    id="new_column_5",
    name="New Column 3",
    type=ColumnType.SELECT.value,
    settings={
        "width": 255,
        "isVisible": True,
        "isEditable": True,
        "options": [
            {"name": "name", "color": "hsl(200, 100%, 50%)", "value": "name"},
            {"name": "age", "color": "hsl(200, 100%, 50%)", "value": "age"},
            {"name": "gender", "color": "hsl(200, 100%, 50%)", "value": "gender"},
        ]
    },
)
new_column

{'id': 'new_column_5',
 'name': 'New Column 5',
 'type': 'select',
 'settings': {'id': 'new_column_5',
  'name': 'New Column 5',
  'type': 'select',
  'width': 255,
  'options': [{'name': 'name', 'value': 'name'},
   {'name': 'age', 'value': 'age'},
   {'name': 'gender', 'value': 'gender'}],
  'isVisible': True,
  'isEditable': True},
 'created_at': '2025-04-16T18:11:14.305975+00:00',
 'updated_at': '2025-04-16T18:11:14.305975+00:00',
 'datatable_id': 'cc6794e1-3505-4d5c-b403-ca7e55142bbc'}

In [ ]:
await client.list_dataset_columns(projects["items"][0]["id"], "271b8bc7-2d04-43b8-8960-ce20365f546b")

{'items': [{'id': 'dQ7hCb1AUfog',
   'name': 'tags_color_coded',
   'type': 'select',
   'settings': {'id': 'dQ7hCb1AUfog',
    'name': 'tags_color_coded',
    'type': 'select',
    'width': 255,
    'options': [{'name': 'red', 'color': 'hsl(0, 85%, 60%)', 'value': 'red'},
     {'name': 'green', 'color': 'hsl(30, 85%, 60%)', 'value': 'green'},
     {'name': 'blue', 'color': 'hsl(45, 85%, 60%)', 'value': 'blue'}],
    'isVisible': True,
    'isEditable': True},
   'created_at': '2025-04-16T19:00:39.936764+00:00',
   'updated_at': '2025-04-16T19:00:39.936764+00:00',
   'datatable_id': '271b8bc7-2d04-43b8-8960-ce20365f546b'},
  {'id': 'eCAiMBRqm0Uc',
   'name': 'id',
   'type': 'number',
   'settings': {'id': 'eCAiMBRqm0Uc',
    'name': 'id',
    'type': 'number',
    'width': 255,
    'isVisible': True,
    'isEditable': True},
   'created_at': '2025-04-16T19:00:39.971857+00:00',
   'updated_at': '2025-04-16T19:00:39.971857+00:00',
   'datatable_id': '271b8bc7-2d04-43b8-8960-ce20365f546b

In [ ]:
col3 = await client.get_dataset_column(
    projects["items"][0]["id"], datasets["id"], "new_column_3"
)
col3

{'id': 'new_column_3',
 'name': 'New Column 3',
 'type': 'text',
 'settings': {'id': 'new_column_3',
  'name': 'New Column 3',
  'type': 'text',
  'max_length': 255,
  'is_required': True},
 'created_at': '2025-04-10T02:22:07.300895+00:00',
 'updated_at': '2025-04-10T02:22:07.300895+00:00',
 'datatable_id': 'ebc3dd3e-f88b-4f8b-8c72-6cfcae0a0cd4'}

In [ ]:
await client.update_dataset_column(
    projects["items"][0]["id"],
    datasets["id"],
    "new_column_3",
    name="New Column 3 Updated",
    type=ColumnType.NUMBER.value,
)

{'id': 'new_column_3',
 'name': 'New Column 3 Updated',
 'type': 'number',
 'settings': {'id': 'new_column_3',
  'name': 'New Column 3',
  'type': 'text',
  'max_length': 255,
  'is_required': True},
 'created_at': '2025-04-10T02:22:07.300895+00:00',
 'updated_at': '2025-04-10T02:22:11.116882+00:00',
 'datatable_id': 'ebc3dd3e-f88b-4f8b-8c72-6cfcae0a0cd4'}

In [ ]:
await client.delete_dataset_column(
    projects["items"][0]["id"], datasets["id"], "new_column_3"
)

### Rows (for datasets)

In [ ]:
#| export
#---- Dataset Rows ----
@patch
async def list_dataset_rows(
    self: RagasApiClient,
    project_id: str,
    dataset_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List rows in a dataset."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/datasets/{dataset_id}/rows", **params
    )

@patch
async def get_dataset_row(
    self: RagasApiClient, project_id: str, dataset_id: str, row_id: str
) -> t.Dict:
    """Get a specific row in a dataset."""
    return await self._get_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}"
    )

@patch
async def create_dataset_row(
    self: RagasApiClient, project_id: str, dataset_id: str, id: str, data: t.Dict
) -> t.Dict:
    """Create a new row in a dataset."""
    row_data = {"id": id, "data": data}
    return await self._create_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows", row_data
    )

@patch
async def update_dataset_row(
    self: RagasApiClient, project_id: str, dataset_id: str, row_id: str, data: t.Dict
) -> t.Dict:
    """Update an existing row in a dataset."""
    row_data = {"data": data}
    return await self._update_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}",
        row_data,
    )

@patch
async def delete_dataset_row(
    self: RagasApiClient, project_id: str, dataset_id: str, row_id: str
) -> None:
    """Delete a row from a dataset."""
    await self._delete_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}"
    )


In [ ]:
datasets["id"]

'3374b891-8398-41bd-8f81-2867759df294'

In [ ]:
await client.create_dataset_row(
    project_id=projects["items"][0]["id"],
    dataset_id=datasets["id"],
    id="",
    data={"new_column_3": "name"},
)


{'id': '',
 'data': {'id': '', 'new_column_3': 'name'},
 'created_at': '2025-04-16T17:46:39.100525+00:00',
 'updated_at': '2025-04-16T17:46:39.100525+00:00',
 'datatable_id': '3374b891-8398-41bd-8f81-2867759df294'}

### Get a Dataset Visualized - Created From UI
Lets Create a new dataset and add columns and rows via the endpoint to see how it behaves

In [ ]:
# generate a dataset
dataset = await client.create_dataset(
    project_id=TEST_PROJECT_ID,
    name="Dataset Visualized from UI",
    description="This is a dataset created from the UI",
)

# show url
WEB_ENDPOINT = "https://dev.app.ragas.io"
url = f"{WEB_ENDPOINT}/dashboard/projects/{TEST_PROJECT_ID}/datasets/{dataset['id']}"
url

'https://dev.app.ragas.io/dashboard/projects/e1b3f1e4-d344-48f4-a178-84e7e32e6ab6/datasets/dbccf6aa-b923-47ed-8e97-bd46f2f2cee8'

In [ ]:
# list columns
columns = await client.list_dataset_columns(TEST_PROJECT_ID, dataset["id"])
# list rows
rows = await client.list_dataset_rows(TEST_PROJECT_ID, dataset["id"])

In [ ]:
columns


{'items': [],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 0,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

In [ ]:
rows

{'items': [],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 0,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

### Create a Dataset from data

we want to be able to use the API with python data like this `t.List[t.Dict]`.
```py
# how we want the data to look
data = [
    {
        "id": "1",
        "query": "What is the capital of France?",
        "persona": "John",
        "ground_truth": "Paris",
    },
    {
        "id": "2",
        "query": "What is the capital of Germany?",
        "persona": "Jane",
        "ground_truth": "Berlin",
    },
    {
        "id": "3",
        "query": "What is the capital of Italy?",
        "persona": "John",
        "ground_truth": "Rome",
    },
]
```

In [ ]:
# print out column types
print([col.value for col in ColumnType])

['number', 'text', 'longText', 'select', 'date', 'multiSelect', 'checkbox', 'custom']


In [ ]:
# it should be able to handle simple python dicts
data = [
    {
        "id": "1",
        "query": "What is the capital of France?",
        "persona": "John",
        "ground_truth": "Paris",
    },
    {
        "id": "2",
        "query": "What is the capital of Germany?",
        "persona": "Jane",
        "ground_truth": "Berlin",
    },
]

There can be 2 ways to pass in data

1. Data can come as either as simple dicts

```py
data = [
    {"column_1": "value", "column_2": "value"}
]
```

2. or if you want to give more settings

```py
data = [
    {
        "column_1": {"data": "value", "type": ColumnType.text},
        "column_2": {"data": "value", "type": ColumnType.number},
    }
]
```

3. after that you will have to pass a list `Column` and `Row` to add it.

In [ ]:
# test data
test_data_columns = [
    {"name": "id", "type": ColumnType.NUMBER.value},
    {"name": "query", "type": ColumnType.TEXT.value},
    {"name": "persona", "type": ColumnType.TEXT.value},
    {"name": "ground_truth", "type": ColumnType.TEXT.value},
]

test_data_rows = [{
    "id": "1",
    "query": "What is the capital of France?",
    "persona": "John",
    "ground_truth": "Paris",
}, {
    "id": "2",
    "query": "What is the capital of Germany?",
    "persona": "Jane",
    "ground_truth": "Berlin",
}, {
    "id": "3",
    "query": "What is the capital of Italy?",
    "persona": "John",
    "ground_truth": "Rome",
}]



In [ ]:
#| export
import uuid
import string

In [ ]:
#| export
def create_nano_id(size=12):
    # Define characters to use (alphanumeric)
    alphabet = string.ascii_letters + string.digits
    
    # Generate UUID and convert to int
    uuid_int = uuid.uuid4().int
    
    # Convert to base62
    result = ""
    while uuid_int:
        uuid_int, remainder = divmod(uuid_int, len(alphabet))
        result = alphabet[remainder] + result
    
    # Pad if necessary and return desired length
    return result[:size]

In [ ]:
# Usage
nano_id = create_nano_id()  # e.g., "8dK9cNw3mP5x"
nano_id

'Anvz5k9geU7T'

In [ ]:
#| export
import uuid
import string

In [ ]:
#| export
def create_nano_id(size=12):
    # Define characters to use (alphanumeric)
    alphabet = string.ascii_letters + string.digits
    
    # Generate UUID and convert to int
    uuid_int = uuid.uuid4().int
    
    # Convert to base62
    result = ""
    while uuid_int:
        uuid_int, remainder = divmod(uuid_int, len(alphabet))
        result = alphabet[remainder] + result
    
    # Pad if necessary and return desired length
    return result[:size]

In [ ]:
# Usage
nano_id = create_nano_id()  # e.g., "8dK9cNw3mP5x"
nano_id

'Anvz5k9geU7T'

In [ ]:
#| export
# Default settings for columns
DEFAULT_SETTINGS = {
    "is_required": False,
    "max_length": 1000
}

# Model definitions
class Column(BaseModel):
    id: str = Field(default_factory=create_nano_id)
    name: str = Field(...)
    type: str = Field(...)
    settings: t.Dict = Field(default_factory=lambda: DEFAULT_SETTINGS.copy())
    col_order: t.Optional[int] = Field(default=None)

class RowCell(BaseModel):
    data: t.Any = Field(...)
    column_id: str = Field(...)

class Row(BaseModel):
    id: str = Field(default_factory=create_nano_id)
    data: t.List[RowCell] = Field(...)

In [ ]:
#| export
#---- Resource With Data Helper Methods ----
@patch
async def _create_with_data(
    self: RagasApiClient,
    resource_type: str,
    project_id: str,
    name: str, 
    description: str,
    columns: t.List[Column],
    rows: t.List[Row],
    batch_size: int = 50
) -> t.Dict:
    """Generic method to create a resource with columns and rows.
    
    Args:
        resource_type: Type of resource ("dataset" or "experiment")
        project_id: Project ID
        name: Resource name
        description: Resource description
        columns: List of column definitions
        rows: List of row data
        batch_size: Number of operations to perform concurrently
        
    Returns:
        The created resource
    """
    # Select appropriate methods based on resource type
    if resource_type == "dataset":
        create_fn = self.create_dataset
        create_col_fn = self.create_dataset_column
        create_row_fn = self.create_dataset_row
        delete_fn = self.delete_dataset
        id_key = "dataset_id"
    elif resource_type == "experiment":
        create_fn = self.create_experiment
        create_col_fn = self.create_experiment_column
        create_row_fn = self.create_experiment_row
        delete_fn = self.delete_experiment
        id_key = "experiment_id"
    else:
        raise ValueError(f"Unsupported resource type: {resource_type}")
        
    try:
        # Create the resource
        resource = await create_fn(project_id, name, description)
        
        # Process columns in batches
        for i in range(0, len(columns), batch_size):
            batch = columns[i:i+batch_size]
            col_tasks = []
            
            for col in batch:
                params = {
                    "project_id": project_id,
                    id_key: resource["id"], # dataset_id here
                    "id": col.id,
                    "name": col.name,
                    "type": col.type,
                    "settings": col.settings
                }
                if col.col_order is not None:
                    params["col_order"] = col.col_order
                
                col_tasks.append(create_col_fn(**params))
            
            await asyncio.gather(*col_tasks)
            
        # Process rows in batches
        for i in range(0, len(rows), batch_size):
            batch = rows[i:i+batch_size]
            row_tasks = []
            
            for row in batch:
                row_data = {cell.column_id: cell.data for cell in row.data}
                row_tasks.append(
                    create_row_fn(
                        project_id=project_id,
                        **{id_key: resource["id"]},
                        id=row.id,
                        data=row_data
                    )
                )
            
            await asyncio.gather(*row_tasks)
            
        return resource
        
    except Exception as e:
        # Clean up on error
        if 'resource' in locals():
            try:
                await delete_fn(project_id, resource["id"])
            except:
                pass  # Ignore cleanup errors
        raise e

@patch
async def create_dataset_with_data(
    self: RagasApiClient,
    project_id: str,
    name: str,
    description: str,
    columns: t.List[Column],
    rows: t.List[Row],
    batch_size: int = 50
) -> t.Dict:
    """Create a dataset with columns and rows.
    
    This method creates a dataset and populates it with columns and rows in an
    optimized way using concurrent requests.
    
    Args:
        project_id: Project ID
        name: Dataset name
        description: Dataset description
        columns: List of column definitions
        rows: List of row data
        batch_size: Number of operations to perform concurrently
        
    Returns:
        The created dataset
    """
    return await self._create_with_data(
        "dataset", project_id, name, description, columns, rows, batch_size
    )

Now lets test this.

In [ ]:
# Create Column objects
column_objects = []
for col in test_data_columns:
    column_objects.append(Column(
        name=col["name"],
        type=col["type"]
        # id and settings will be auto-generated
    ))

# Create a mapping of column names to their IDs for creating rows
column_map = {col.name: col.id for col in column_objects}

# Create Row objects
row_objects = []
for row in test_data_rows:
    cells = []
    for key, value in row.items():
        if key in column_map:  # Skip any extra fields not in columns
            cells.append(RowCell(
                data=value,
                column_id=column_map[key]
            ))
    row_objects.append(Row(data=cells))

# Now we can create the dataset
dataset = await client.create_dataset_with_data(
    project_id=TEST_PROJECT_ID,
    name="Capitals Dataset",
    description="A dataset about capital cities",
    columns=column_objects,
    rows=row_objects
)

print(f"Created dataset with ID: {dataset['id']}")

# Verify the data
columns = await client.list_dataset_columns(TEST_PROJECT_ID, dataset["id"])
print(f"Created {len(columns['items'])} columns")

rows = await client.list_dataset_rows(TEST_PROJECT_ID, dataset["id"])
print(f"Created {len(rows['items'])} rows")

Created dataset with ID: 5e7912f4-6a65-4d0c-bf79-0fab9ddda40c
Created 4 columns
Created 3 rows


In [ ]:
# get dataset url
url = f"{WEB_ENDPOINT}/dashboard/projects/{TEST_PROJECT_ID}/datasets/{dataset['id']}"
url

'https://dev.app.ragas.io/dashboard/projects/e1b3f1e4-d344-48f4-a178-84e7e32e6ab6/datasets/5e7912f4-6a65-4d0c-bf79-0fab9ddda40c'

In [ ]:
# cleanup
await client.delete_dataset(TEST_PROJECT_ID, dataset["id"])

### The same but for Experiments

In [ ]:
#| export
#---- Experiment Columns ----
@patch
async def list_experiment_columns(
    self: RagasApiClient,
    project_id: str,
    experiment_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List columns in an experiment."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/experiments/{experiment_id}/columns", **params
    )

@patch
async def get_experiment_column(
    self: RagasApiClient, project_id: str, experiment_id: str, column_id: str
) -> t.Dict:
    """Get a specific column in an experiment."""
    return await self._get_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}"
    )

@patch
async def create_experiment_column(
    self: RagasApiClient,
    project_id: str,
    experiment_id: str,
    id: str,
    name: str,
    type: str,
    col_order: t.Optional[int] = None,
    settings: t.Optional[t.Dict] = None,
) -> t.Dict:
    """Create a new column in an experiment."""
    data = {"id": id, "name": name, "type": type}
    if col_order is not None:
        data["col_order"] = col_order
    if settings:
        data["settings"] = settings
    return await self._create_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns", data
    )

@patch
async def update_experiment_column(
    self: RagasApiClient, project_id: str, experiment_id: str, column_id: str, **column_data
) -> t.Dict:
    """Update an existing column in an experiment."""
    return await self._update_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}",
        column_data,
    )

@patch
async def delete_experiment_column(
    self: RagasApiClient, project_id: str, experiment_id: str, column_id: str
) -> None:
    """Delete a column from an experiment."""
    await self._delete_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}"
    )

#---- Experiment Rows ----
@patch
async def list_experiment_rows(
    self: RagasApiClient,
    project_id: str,
    experiment_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List rows in an experiment."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/experiments/{experiment_id}/rows", **params
    )

@patch
async def get_experiment_row(
    self: RagasApiClient, project_id: str, experiment_id: str, row_id: str
) -> t.Dict:
    """Get a specific row in an experiment."""
    return await self._get_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}"
    )

@patch
async def create_experiment_row(
    self: RagasApiClient, project_id: str, experiment_id: str, id: str, data: t.Dict
) -> t.Dict:
    """Create a new row in an experiment."""
    row_data = {"id": id, "data": data}
    return await self._create_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows", row_data
    )

@patch
async def update_experiment_row(
    self: RagasApiClient, project_id: str, experiment_id: str, row_id: str, data: t.Dict
) -> t.Dict:
    """Update an existing row in an experiment."""
    row_data = {"data": data}
    return await self._update_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}",
        row_data,
    )

@patch
async def delete_experiment_row(
    self: RagasApiClient, project_id: str, experiment_id: str, row_id: str
) -> None:
    """Delete a row from an experiment."""
    await self._delete_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}"
    )

In [ ]:
await client.create_experiment(TEST_PROJECT_ID, "New Experiment", "This is a new experiment")

{'id': '7c695b58-7fc3-464c-a18b-a96e35f9684d',
 'name': 'New Experiment',
 'description': 'This is a new experiment',
 'updated_at': '2025-04-09T17:03:44.340782+00:00',
 'created_at': '2025-04-09T17:03:44.340782+00:00',
 'version_counter': 0,
 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}

In [ ]:
experiments = await client.list_experiments(TEST_PROJECT_ID)
EXPERIMENT_ID = experiments["items"][0]["id"]
EXPERIMENT_ID

'78fd6c58-7edf-4239-93d1-4f49185d8e49'

In [ ]:
#| export
@patch
async def create_experiment_with_data(
    self: RagasApiClient,
    project_id: str,
    name: str,
    description: str,
    columns: t.List[Column],
    rows: t.List[Row],
    batch_size: int = 50
) -> t.Dict:
    """Create an experiment with columns and rows.
    
    This method creates an experiment and populates it with columns and rows in an
    optimized way using concurrent requests.
    
    Args:
        project_id: Project ID
        name: Experiment name
        description: Experiment description
        columns: List of column definitions
        rows: List of row data
        batch_size: Number of operations to perform concurrently
        
    Returns:
        The created experiment
    """
    return await self._create_with_data(
        "experiment", project_id, name, description, columns, rows, batch_size
    )

In [ ]:
#| export
#---- Utility Methods ----
@patch
def create_column(
    self: RagasApiClient, 
    name: str, 
    type: str, 
    settings: t.Optional[t.Dict] = None, 
    col_order: t.Optional[int] = None,
    id: t.Optional[str] = None
) -> Column:
    """Create a Column object.
    
    Args:
        name: Column name
        type: Column type (use ColumnType enum)
        settings: Column settings
        col_order: Column order
        id: Custom ID (generates one if not provided)
        
    Returns:
        Column object
    """
    params = {"name": name, "type": type}
    if settings:
        params["settings"] = settings
    if col_order is not None:
        params["col_order"] = col_order
    if id:
        params["id"] = id
        
    return Column(**params)
    
@patch
def create_row(
    self: RagasApiClient, 
    data: t.Dict[str, t.Any], 
    column_map: t.Dict[str, str],
    id: t.Optional[str] = None
) -> Row:
    """Create a Row object from a dictionary.
    
    Args:
        data: Dictionary mapping column names to values
        column_map: Dictionary mapping column names to column IDs
        id: Custom ID (generates one if not provided)
        
    Returns:
        Row object
    """
    cells = []
    for col_name, value in data.items():
        if col_name in column_map:
            cells.append(RowCell(
                data=value,
                column_id=column_map[col_name]
            ))
            
    params = {"data": cells}
    if id:
        params["id"] = id
        
    return Row(**params)
    
@patch
def create_column_map(self: RagasApiClient, columns: t.List[Column]) -> t.Dict[str, str]:
    """Create a mapping of column names to IDs.
    
    Args:
        columns: List of column objects
        
    Returns:
        Dictionary mapping column names to IDs
    """
    return {col.name: col.id for col in columns}
    
@patch
async def convert_raw_data(
    self: RagasApiClient,
    column_defs: t.List[t.Dict],
    row_data: t.List[t.Dict]
) -> t.Tuple[t.List[Column], t.List[Row]]:
    """Convert raw data to column and row objects.
    
    Args:
        column_defs: List of column definitions (dicts with name, type)
        row_data: List of dictionaries with row data
        
    Returns:
        Tuple of (columns, rows)
    """
    # Create columns
    columns = []
    for col in column_defs:
        columns.append(self.create_column(**col))
        
    # Create column map
    column_map = self.create_column_map(columns)
    
    # Create rows
    rows = []
    for data in row_data:
        rows.append(self.create_row(data, column_map))
        
    return columns, rows